In [ ]:
import re
import pandas as pd

In [ ]:
! pip install sqlparse

#### Ignore this block

In [ ]:
# Read the Excel file
excel_file = "./select_cols.xlsx"
df = pd.read_excel(excel_file)

# Initialize an empty list to store unmatched columns for each row
all_unmatched_columns = []

df_len = len(df)
# Iterate through the first 10 rows (adjust the range if needed)
for i in range(df_len):
    # Extract SQL statements from the correct_query and temp_0 columns
    sql1 = df['temp_0'].iloc[i]
    sql2 = df['correct_query'].iloc[i]
    
    if sql1 == "not_evaluated":
        print(f"Row {i+1}:Not evaluated")
        continue
    
    # Extract column names using regex
    pattern = r"SELECT\s+(.*?)\s+FROM"
    columns_sql1 = re.search(pattern, sql1, re.IGNORECASE).group(1)
    columns_sql2 = re.search(pattern, sql2, re.IGNORECASE).group(1)
    
    # Create sets of column names
    set_columns_sql1 = set(map(str.strip, columns_sql1.split(',')))
    set_columns_sql2 = set(map(str.strip, columns_sql2.split(',')))
    
    # Find unmatched columns for this row
    unmatched_columns = set_columns_sql1 - set_columns_sql2
    
    # Append unmatched columns to the list
    all_unmatched_columns.append(unmatched_columns)

# Print unmatched columns for each row
for i, unmatched_columns in enumerate(all_unmatched_columns):
    print(f"Unmatched Columns for Row {i + 1}:", unmatched_columns)

In [ ]:
def get_missing_columns(sql_query, column_list):

    # Extract column names from the SELECT statement using regex
    pattern = r'SELECT\s+(.+?)\s+FROM'
    match = re.search(pattern, sql_query, re.IGNORECASE)
    
    print("SQL query:", sql_query)
    selected_columns = match.group(1).split(',')

    # Process each selected column
    valid_columns = []
    for col in selected_columns:
        # Handle functions and aliases using regex
        col = re.sub(r'\s+AS\s+\w+', '', col, flags=re.IGNORECASE).strip()
        col = re.sub(r'[^()\s]*\(', '', col).strip()  # Remove functions if present
        col = col.strip('()')  # Remove parentheses if present
        col = re.sub(r'^\s*DISTINCT\s+', '', col, flags=re.IGNORECASE)  # Remove DISTINCT if present
        
        
        if col.strip() not in ['*', "'EEEE'", '12', '2', "'%W'"]:
            valid_columns.append(col.strip())

    print("SQL columns:", valid_columns)

    # Check if selected columns are in the list
    missing_columns = [col for col in valid_columns if col not in column_list]
    
    if not missing_columns:
        return valid_columns, None, 0
    else:
        return valid_columns, missing_columns, 1


In [ ]:
# Read the Excel file
col_excel_file = "./test_cols.xlsx"
col_df = pd.read_excel(col_excel_file)

# Initialize an empty list to store unmatched columns for each row
all_unmatched_columns = []
generated_sql_queries = []
selected_columns = []
missing_columns = []
is_missing_columns = []

col_df_len = len(col_df)


# Process each input string
for i in range(col_df_len):
    input_string = col_df['temp_col_0.6'].iloc[i]
    sql2 = col_df['correct_query'].iloc[i]

    if input_string == "not_evaluated" or sql2 == "not_evaluated":
        # print(f"Row {i+1}:Not evaluated")
        all_unmatched_columns.append("not_evaluated")
        generated_sql_queries.append("not_evaluated")
        selected_columns.append("not_evaluated")
        missing_columns.append("not_evaluated")
        is_missing_columns.append(0)
    else:
        print(f"Row {i+1}")
        # Find the position of the SQL query
        query_start = input_string.index("SELECT")

        # Extract the list part
        list_part = input_string[:query_start].strip()

        # Extract the SQL query part
        sql_query_part = input_string[query_start:]

        # Remove the surrounding square brackets and single quotes from the list
        column_list = [column.strip("' ") for column in list_part.strip("[]").split(',')]

        generated_sql_queries.append(sql_query_part)

        #Extract missing correct columns from the embedding list
        valid_cols, missing_cols, is_missing = get_missing_columns(sql2, list_part)

        # Extract column names using regex
        pattern = r"SELECT\s+(.*?)\s+FROM"
        columns_sql1 = re.search(pattern, sql_query_part, re.IGNORECASE).group(1)
        columns_sql2 = re.search(pattern, sql2, re.IGNORECASE).group(1)
        
        # Create sets of column names
        set_columns_sql1 = set(map(str.strip, columns_sql1.split(',')))
        set_columns_sql2 = set(map(str.strip, columns_sql2.split(',')))
        
        # Find unmatched columns for this row
        unmatched_columns = set_columns_sql1 - set_columns_sql2
        
        # Append unmatched columns to the list
        all_unmatched_columns.append(unmatched_columns) if unmatched_columns else all_unmatched_columns.append("None")
        selected_columns.append(list_part)
        missing_columns.append(missing_cols) if missing_cols else missing_columns.append("None")
        is_missing_columns.append(is_missing)

# Add a new column "Unmatched_Columns" to the DataFrame
# col_df["Unmatched_Columns"] = all_unmatched_columns
col_df["temp_0.6"] = generated_sql_queries
col_df["Embedding_cols"] = selected_columns
col_df["Missing_cols"] = missing_columns
col_df["is_missing"] = is_missing_columns

# Save the updated DataFrame to the same Excel file
col_df.to_excel(col_excel_file, index=False, sheet_name="temp_0.6")


# Print unmatched columns for each row
# for i, unmatched_columns in enumerate(all_unmatched_columns):
#     print(f"Unmatched Columns for Row {i + 1}:", unmatched_columns)
